In [1]:

import pandas as pd
import re #regex library to format fast to read into pd
import os

import matplotlib.pyplot as plt
import feather

import numpy as np
import platform
import mne


from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test
from mne.stats import (ttest_1samp_no_p, bonferroni_correction, fdr_correction,


                       permutation_t_test, permutation_cluster_1samp_test,spatio_temporal_cluster_1samp_test)
import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    #sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    #sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

from base.files_in_out import getListOfFiles,GetFiles
from stats.stats_helper import filter_list, tTest_ana
import stats.t_tests_cluster as clus
import stats.stats_helper as s_hp
import base.base_constants as cs
from stats.stats_helper import filter_list, tTest_ana, filter_list_equal

C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [3]:
def data_cutter(data, time_limits=[-0.3, 0.8], new_time_limits=[0.2, 0.6], sfreq=256):
    """
    Cuts a 3D EEG data array ( time x electrodes) based on specified time limits.
    Parameters:
        data (np.ndarray): EEG data of shape (electrodes, time)
        time_limits (list of float): Original epoch time range in seconds (e.g., [-0.3, 0.8])
        new_time_limits (list of float): Desired time range to extract (e.g., [0.2, 0.6])
        sfreq (int): Sampling frequency in Hz (default is 256)
    Returns:
        np.ndarray: The cut data with shape (new_time_points,electrodes)
    """
    # Calculate the sample indices corresponding to the new time limits
    start_idx = int(np.round((new_time_limits[0] - time_limits[0]) * sfreq))
    end_idx = int(np.round((new_time_limits[1] - time_limits[0]) * sfreq))
    # Cut the data along the time dimension (axis=1)
    return data[start_idx:end_idx,:]



In [4]:
def get_evo(data,tmin=-0.1):
    biosemi_montage = mne.channels.make_standard_montage('biosemi128')

    info = mne.create_info(ch_names=biosemi_montage.ch_names, sfreq=256.,
                           ch_types='eeg')
    evok=mne.EvokedArray(data,info,tmin=tmin)
    evok.set_montage(biosemi_montage)
    return evok

# DF to perform ROI analysis
the actual analysis are on R in the file ROI_unfold_hbc

In [5]:
filepath='Z:/BBC\WP1\data\EEG\int/ana/behav/group_n_int_acc_df.feather'
hbc_df=pd.read_feather(filepath)

In [6]:
med=hbc_df['mean_int'].median()
hbc_df['h_per'] = (hbc_df['mean_int'] >= med).astype(int)

In [7]:
hbc_df=hbc_df[['g_num','mean_int','h_per']]

## aware vs unaware in aware fit

In [9]:
regions=dict(dc_post=['A5','A14','A16','A18','A23'],dc_front=['C32','D4'],nodc=['B12','B14','B23','B24'] )

erp_type='hep'
modtype='awafit'
hbc_cond='low'
if hbc_cond=='high':
    subject_list=g_num_high
elif hbc_cond=='low':
    subject_list=g_num_low

In [41]:

subject_list=cs.G_N
awareness_list=['aware','unaware']
dc_list=['dc','nodc']


all_results=[]

for awareness_cond in awareness_list:
    for dc_cond in dc_list:
        for idx,g_num in enumerate(subject_list):
            subject_results={}
            subject_results['g_num']=g_num
            file_dir=f'ana/deconvolution/ep_betas/awafit/{dc_cond}/{g_num}_{awareness_cond}_hep__Intercept__{dc_cond}.ep'
            data=np.loadtxt(file_dir)
            data=data_cutter(data,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])
            subject_results['awareness']=awareness_cond
            subject_results['dc']=dc_cond

            for region in regions.keys():
                evo_1=get_evo(data.T)
                evo_1.pick_channels(regions[region])
                data_erp=evo_1.data
                data_region=data_cutter(data_erp.T,time_limits=[-0.1,0.7],new_time_limits=[0.49,0.52])
                subject_results[region]=data_region.mean()

            all_results.append(subject_results)

            df=pd.DataFrame(all_results)



In [42]:
df

,g_num,awareness,dc,dc_post,dc_front,nodc
0,g01,aware,dc,-0.592519,1.204980,-0.249928
1,g03,aware,dc,-0.777534,0.751541,-0.702656
2,g08,aware,dc,-0.245768,0.781978,-0.172146
3,g10,aware,dc,-0.169978,0.312345,-0.353291
4,g11,aware,dc,-1.237423,0.875180,0.073668
...,...,...,...,...,...,...
115,g46,unaware,nodc,-0.758388,0.380957,0.948056
116,g47,unaware,nodc,0.163072,0.309736,0.101673
117,g49,unaware,nodc,0.469250,-0.719561,-0.167920
118,g51,unaware,nodc,-0.006452,-0.241782,-0.073540


In [43]:
df_final=df.merge(hbc_df,on='g_num')

In [44]:
df_final

,g_num,awareness,dc,dc_post,dc_front,nodc,mean_int,h_per
0,g01,aware,dc,-0.592519,1.204980,-0.249928,0.489818,1
1,g01,aware,nodc,-0.474549,1.117957,-0.575139,0.489818,1
2,g01,unaware,dc,-0.392548,0.993822,-0.201834,0.489818,1
3,g01,unaware,nodc,-0.467532,1.116099,-0.524208,0.489818,1
4,g03,aware,dc,-0.777534,0.751541,-0.702656,0.402878,0
...,...,...,...,...,...,...,...,...
115,g51,unaware,nodc,-0.006452,-0.241782,-0.073540,0.333747,0
116,g52,aware,dc,-0.175531,0.399921,-0.973764,0.017568,0
117,g52,aware,nodc,0.196500,0.371109,-0.742729,0.017568,0
118,g52,unaware,dc,0.086484,-0.070891,-0.460125,0.017568,0


In [46]:
file_dir=f'ana/deconvolution/stats/ROI_df_int.csv'
df_final.to_csv(file_dir)